<a href="https://colab.research.google.com/github/IcarusFe1l/ArkadasimHG/blob/main/Task7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dfRet =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
dfRet.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [ ]:
# Fill missing Product_Category_2 and Product_Category_3 values with the median
dfRet['Product_Category_2'].fillna(dfRet['Product_Category_2'].median(), inplace=True)
dfRet['Product_Category_3'].fillna(dfRet['Product_Category_3'].median(), inplace=True)


In [ ]:
# Create a user-product interaction matrix
user_product_matrix = dfRet.pivot_table(index='User_ID', columns='Product_ID', values='Purchase', aggfunc='sum', fill_value=0)

# Convert the matrix to a sparse matrix format
user_product_sparse_matrix = csr_matrix(user_product_matrix.values)

In [ ]:
# Apply NearestNeighbors algorithm
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model_knn.fit(user_product_sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [ ]:
# Function to get product bundle recommendations
def get_product_bundle_recommendations(product_id, n_recommendations=5):
    product_index = user_product_matrix.columns.get_loc(product_id)
    # Remove the transpose operation here as the model is trained on the original matrix
    distances, indices = model_knn.kneighbors(user_product_sparse_matrix[product_index], n_neighbors=n_recommendations+1)

    recommendations = []
    for i in range(1, len(distances.flatten())):
        recommendations.append(user_product_matrix.columns[indices.flatten()[i]])

    return recommendations


In [ ]:
# Test the function with a sample product_id
sample_product_id = 'P00265242'
recommendations = get_product_bundle_recommendations(sample_product_id, n_recommendations=5)

print(f"Products frequently bought with {sample_product_id}: {recommendations}")

Products frequently bought with P00265242: ['P00268542', 'P00271442', 'P00302842', 'P00288542', 'P00182942']


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df.shape


(550068, 12)

In [ ]:
df_idbased = pd.DataFrame(columns=["User_ID", "Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status", "Total_Purchase"])
df_idbased = df.groupby("User_ID").agg({"Gender":"first", "Age":"first", "Occupation":"first", "City_Category":"first", "Stay_In_Current_City_Years":"first", "Marital_Status":"first", "Purchase":"sum"})
df_idbased.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Purchase
User_ID,,,,,,,
1000001,F,0-17,10,A,2,0,334093
1000002,M,55+,16,C,4+,0,810472
1000003,M,26-35,15,A,3,0,341635
1000004,M,46-50,7,B,2,1,206468
1000005,M,26-35,20,A,1,1,821001


In [ ]:
df['clean_description'] = df['Product_ID']
dfList =df.groupby('User_ID').clean_description.apply(list)
dfList

User_ID
1000001    [P00069042, P00248942, P00087842, P00085442, P...
1000002    [P00285442, P00112842, P00293242, P00289342, P...
1000003    [P00193542, P00132842, P0098342, P00010242, P0...
1000004    [P00184942, P00346142, P0097242, P00046742, P0...
1000005    [P00274942, P00251242, P00014542, P00031342, P...
                                 ...                        
1006036    [P00237642, P00250342, P00173242, P00255342, P...
1006037    [P00067342, P00111842, P00323642, P00177442, P...
1006038    [P00034742, P00251242, P00086042, P00109542, P...
1006039    [P00114042, P00199442, P00088542, P00254242, P...
1006040    [P00184042, P00158742, P00001742, P00148642, P...
Name: clean_description, Length: 5891, dtype: object

In [ ]:
!pip3 install mlxtend --upgrade
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.22.0
    Uninstalling mlxtend-0.22.0:
      Successfully uninstalled mlxtend-0.22.0


In [ ]:
%time
te = TransactionEncoder()
te_ary = te.fit(dfList).transform(dfList)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

df_encoded

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,P00001142,P00001242,P00001342,P00001442,P00001542,P00001642,P00001742,P00001842,P00001942,P00002042,P00002142,P00002242,P00002342,P00002442,P00002542,P00002642,P00002742,P00002842,P00002942,P00003042,P00003142,P00003242,P00003342,P00003442,P00003542,P00003642,P00003742,P00003842,P00003942,P00004042,P00004142,P00004242,P00004342,P00004442,P00004542,P00004642,P00004742,P00004842,P00004942,P00005042,...,P0094942,P0095042,P0095142,P0095242,P0095342,P0095442,P0095542,P0095642,P0095742,P0095842,P0095942,P0096042,P0096142,P0096242,P0096342,P0096442,P0096542,P0096642,P0096742,P0096842,P0096942,P0097042,P0097142,P0097242,P0097342,P0097442,P0097542,P0097642,P0097742,P0097842,P0097942,P0098042,P0098142,P0098242,P0098342,P0098442,P0098542,P0098642,P0098742,P0098842,P0098942,P0099042,P0099142,P0099242,P0099342,P0099442,P0099642,P0099742,P0099842,P0099942
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,.

In [ ]:
%time
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


,support,itemsets
0,0.195553,(P00000142)
1,0.063826,(P00000242)
2,0.086912,(P00000642)
3,0.085384,(P00001042)
4,0.098116,(P00001142)
...,...,...
2143,0.050755,"(P00355142, P00334242)"
2144,0.050416,"(P00110742, P00025442, P00057642)"
2145,0.054660,"(P00110742, P00025442, P00112142)"
2146,0.050246,"(P00110742, P00025442, P00237542)"


In [ ]:

apriori_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.55)
apriori_rules.sort_values(by='confidence', ascending=False, inplace=True)
apriori_rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(P00145042, P00270942)",(P00057642),0.090816,0.249533,0.051095,0.562617,2.254677,0.028433,1.715811,0.612063


In this sheet I used 2 methmods one was from task7 one was from the IE303 integer optimization course of ours. Neigherst neighbour works pretty well on these kind of suggestion methods since it works according to relation with other items.